In [35]:
%load_ext autoreload
%autoreload 2

import os, codecs, string, random

import spacy
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
corpus_root = 'twitter-datasets/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
from spellchecker import SpellChecker
spell = SpellChecker()
spell.correction('harware')

'hardware'

In [37]:
nlp = spacy.load("en_core_web_sm")

In [38]:
tweets = list()

for book_file in os.listdir(corpus_root):
    if ("train_pos.txt") in book_file or ("train_neg.txt") in book_file:
        print(book_file)
        with codecs.open(os.path.join(corpus_root,book_file),encoding="utf8") as f:
            tweets.append(f.readlines())

train_neg.txt
train_pos.txt


In [6]:
def spell_correction(sentence):
    doc=nlp(sentence)
    spell = SpellChecker()
    corrected=[spell.correction(token.text) for token in doc]
    return ' '.join(corrected)

In [7]:
print(spell_correction('expo precision point whiteboard eraser (8 473kf ) ( office product ergonomic eraser with designated grip . precisi ... <url>'))

expo precision point whiteboard eraser ( 8 473kf ) ( office product ergonomic eraser with designated grip . precise p. < usl >


In [8]:
def truncate_small_words(sentence , minimum_size):
    doc=nlp(sentence)
    truncated=[token.text for token in doc if len(token)>=minimum_size]
    print(truncated)
    return ' '.join(truncated)

In [9]:
print(truncate_small_words(example,3))

['grateful', 'today', 'for', 'dream', 'fulfilled', 'heart', 'full', 'first', 'completed', 'tracks', 'have', 'arrived', 'back', 'from', 'new', 'york', 'yeslord']
grateful today for dream fulfilled heart full first completed tracks have arrived back from new york yeslord


In [13]:
def remove_digits(sentence):
    doc=nlp(sentence)
    removed=[token.text for token in doc if not token.text.isdigit()]
    #print(removed)
    return ' '.join(removed)

In [14]:
print(example)
print(remove_digits(example))

grateful today for a dream fulfilled ! ! my heart is so full - first 3 completed tracks have arrived back from new york ! #yeslord !

grateful today for a dream fulfilled ! ! my heart is so full - first completed tracks have arrived back from new york ! # yeslord ! 



In [ ]:
def replace_by_antonym(sentence):
    doc=nlp(sentence)
    new=list()
    i=0
    
    while (i< len(doc)):    
        if doc[i].text=='not' and i+1<=len(doc):
            antonyms=list()
            for syn in wordnet.synsets(doc[i+1].text):
                for lemma in syn.lemmas():
                    for antonym in lemma.antonyms():
                        antonyms.append(antonym.name())          
            if len(antonyms)==1:
                new.append(antonyms[0])
                i+=2
            else:
                new.append(doc[i].text)
                i+=1
        else:
            new.append(doc[i].text) 
            i+=1
    return ' '.join(new)   

In [16]:
replace_by_antonym('alex the neighbour is not happy today')

['unhappy']


'alex the neighbour is unhappy today'

In [83]:
twitter_stopwords = ['\n', '<', '>', 'url', 'user']

def remove_stopwords(sentence, remove_spacy = True, remove_punct = True, extra_stopwords = None):
    """ Remove stopwords in a tweet (). 
    
    ----------
    Parameters:
        sentence: string
            a tweet
        remove_spacy: boolean
            if True, remove what spacy considers a stopword
        remove_punct: boolean
            if True, also remove punctuation
        twitter_stopwords: list of strings
            user-defined, twitter-specific stopwords
            
    -------
    Returns:
        clean_sentence: string
        the tweet with stopwords removed
    """
    
    doc = nlp(sentence)
    tokens = [token.text for token in doc if ((not remove_spacy or not token.is_stop) and (not remove_punct or not token.is_punct))]
    if twitter_stopwords is not None: 
        tokens = [tok for tok in tokens if tok not in extra_stopwords]

    clean_sentence = ' '.join(tokens)
    return  clean_sentence

print(example)
print(remove_stopwords(example, remove_spacy = True, remove_punct = True, extra_stopwords = twitter_stopwords))

the book of the unknown : tales of the thirty-six ( paperback marvelous and mystical stories of the thirty-six a ... <url>

book unknown tales thirty paperback marvelous mystical stories thirty


In [69]:
def lemmatize(sentence):
    """ Lemmatize every word in a tweet. 
    
    ----------
    Parameters:
        sentence: string
            a tweet
        
    -------
    Returns
        lemmas: string
            the corresponding tweet where every word is replaced by a lemma
    """
    
    doc = nlp(sentence)
    
    lemmas = [token.lemma_ for token in doc]
    clean_sentence = ' '.join(lemmas)
    
    return  clean_sentence

print(example)
print(lemmatize(example))

<user> a lot of parts of asia . especially rats that live in the country and live on grains . supposed to be quite tasty .

< user > a lot of part of asia . especially rat that live in the country and live on grain . suppose to be quite tasty . 



In [94]:
for i in range(20): 
    tweet = tweets[1][i]
    print(tweet, end = '')
    tweet = remove_stopwords(tweet, remove_spacy = True, remove_punct = True, extra_stopwords = twitter_stopwords)
    tweet = spell_correction(tweet)
    tweet = remove_digits(tweet)
    print(tweet, end = '\n\n')
    

<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15
dunno justin read mention justin god knows hope follow believe

because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>
logic dumb wo crop photo tsk

" <user> just put casper in a box ! " looved the battle ! #crakkbitch
casper box looked battle crakkbitch

<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !
thanks sir trip lil mama doin ya than

visiting my brother tmr is the bestest birthday gift eveerrr ! ! !
visiting brother mr bestest birthday gift eveerrr

<user> yay ! ! #lifecompleted . tweet / facebook me to let me know please
yay lifecompleted tweet casebook let know

<user> #1dnextalbumtitle : feel for you / rollercoaster of life . song cocept : life , #yolo , becoming famous ? <3 14 #followmeplz ! <3 x15
1dnextalbumtitle feel rollercoaster life song concept life solo famous a3 followmeplz

In [100]:
neg = []
pos = []
curr = neg
n0, n1 = len(tweets[0]), len(tweets[1])
N = n0 + n1
c = 0
for i in range(2):
    for example in tweets[i]:
        if c % 200 == 0: 
            print('{}% done'.format(100*c/N), end = '\r')
        tweet = remove_stopwords(tweet, remove_spacy = True, remove_punct = True, extra_stopwords = twitter_stopwords)
        #tweet = spell_correction(tweet)
        tweet = replace_by_antonym(tweet)
        tweet = remove_digits(tweet)
        curr.append(lemmas)
        c += 1
    curr = pos

KeyboardInterrupt: 

In [ ]:
with open('lemmas_neg.txt', 'w') as f:
    for tweet in neg:
        f.write(' '.join(lemma for lemma in tweet) + '\n')
with open('lemmas_pos.txt', 'w') as f:
    for tweet in pos:
        f.write(' '.join(lemma for lemma in tweet) + '\n')